# Amazon Forecast

In this notebook, we will use Amazon Forecast, a fully managed service for producing time series forecasts using machine learning. 

We will call Forecast APIs using SageMaker for which we need to ensure that the SageMaker role associated with this Notebook environment has the AmazonForecastFullAccess policy attached to it. Please go to the IAM console and check to make sure that the role associated with the notebook has this policy attached.

We also need to ensure that Forecast can access data in S3 buckets. To ensure this, in the IAM console, create a role called **Forecastbasicrole** which has AmazonS3FullAccess policy attached to it.



## Import Libraries and load data

In [ ]:
import json
import sys
import os
import copy
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import time
import datetime as dt
from time import sleep
import boto3
import sagemaker
# importing forecast notebook utility from notebooks/ directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util

**Important:** This requires boto version > 1.12.39. Let's check this. If not, you will need to upgrade your boto version to continue

In [ ]:
if boto3.__version__ > '1.12.39':
    pass
else:
    raise ValueError('boto3 needs to be upgraded to be later than 1.12.39. Consider running !pip install --upgrade boto3')

# Permissions and environment variables

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. You can create a new S3 bucket or use an existing S3 bucket. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Please double-check in the IAM console that you created a role called Forecastbasicrole which has AmazonS3FullAccess policy attached to it.

In [ ]:
text_widget_account_num = util.create_text_widget( "ACCOUNT_NUM", "input your 12 digit account number" )
text_widget_bucket = util.create_text_widget( "bucket", "input your S3 bucket name" )
text_widget_region_name = util.create_text_widget( "REGION_NAME", "input region name.", default_value="us-east-1" )

In [ ]:
ACCOUNT_NUM = text_widget_account_num.value
assert ACCOUNT_NUM, "ACCOUNT_NUM not set."

REGION_NAME = text_widget_region_name.value
assert REGION_NAME, "REGION_NAME not set."

bucket = text_widget_bucket.value
assert bucket, "bucket name not set."

In [ ]:
prefix = 'web-forecast-data' #modify with your preferred prefix
role_arn = 'arn:aws:iam::{}:role/Forecastbasicrole'.format(ACCOUNT_NUM) # Create this role in IAM. Role is needed to get permissions for Forecast to access S3.

In [ ]:
# make sure that the region your bucket is in is the region of the session
session = boto3.Session(region_name=REGION_NAME)
forecast = session.client(service_name='forecast')
s3 = session.client('s3')
forecastquery = session.client(service_name='forecastquery')

In [ ]:
df = pd.read_csv('data/preprocessed_data.csv', parse_dates=True)
df.head()

In [ ]:
df = df.drop(columns = ['Headline'])

In [ ]:
df['PublishDate'] = pd.to_datetime(df['PublishDate'])
df = df.set_index('PublishDate')
df.index = df.index.to_period('1D').to_timestamp()
df.head()

Aggregate the data on a daily basis

In [ ]:
agg_df = pd.DataFrame()
topics = [0, 1, 2, 3]
for topic in topics:
    tdf = df[df['Topic']==topic]
    tdf = tdf.drop(columns = ['Topic'])
    tdf = tdf.resample('1D').mean().fillna(0)
    itemid = np.full(len(tdf), topic)
    tdf['Topic']=itemid
    agg_df = pd.concat([tdf, agg_df], axis=0)
agg_df.head()
print("Shape of final Dataframe for Forecasting = {}".format(agg_df.shape))


In [ ]:
df = agg_df.copy()
df.head()
print(len(df))

In [ ]:
DATASET_FREQUENCY = "D" 
TIMESTAMP_FORMAT = "yyyy-MM-dd"
start_training = pd.Timestamp("2015-11-01", freq = DATASET_FREQUENCY) + pd.Timedelta(days=1)
end_training = pd.Timestamp("2016-06-21", freq = DATASET_FREQUENCY)

# End date for ground truth values to be used in comparison with forecasted values
# (given we are predicting 15 days into the future, this subset will be 15 days past the end_training date)
end_GT = pd.Timestamp("2016-07-05", freq = DATASET_FREQUENCY)

## Create Target and Related Time series

We want to forecast the Facebook ratings for each of the 4 topics in the Topic column of the dataset. In Amazon Forecast, we need to define a target time series which consists of the item id, time stamp and the value we wish to forecast. 

Additionally, we can provide a related time series which can include up to 13 dynamical features, which in our case are the HeadlineSentiment and the topic vectors. Since we can only choose 13 features in Amazon Forecast, we choose 10 out of the 20 topic vectors to illustrate buildng the Forecast model.

As before, we start forecasting from 2015-11-01 and end our training data at 2016-06-21. Using this, we will forecast for 15 days out into the future. 

In [ ]:
# Create Target and Related Time series
target_df = pd.DataFrame()
target_df['item_id'] = df.Topic
target_df['timestamp'] = df.index
target_df['value'] = df.Facebook

In [ ]:
target_df = target_df[(target_df['timestamp']<end_training)&(target_df['timestamp']>start_training)]

In [ ]:
target_df.head()


In [ ]:
# Note: Related time series only takes up to 13 features. 
related_df = pd.DataFrame()
related_df['item_id'] = df.Topic
related_df['timestamp'] = df.index
related_df['SentimentHeadline'] = df.SentimentHeadline
for i in range(10):
    related_df['Headline_{}'.format(i)] = df['Headline_Topic_{}'.format(i)]
related_df.head()

In [ ]:
related_df = related_df[(related_df['timestamp']>start_training)]

## Upload the Target and Related timeseries data to S3

In [ ]:
outdir = './forecast-data'
if not os.path.exists(outdir):
    os.mkdir(outdir)

target_df.to_csv(os.path.join(outdir, 'target_time_series.csv').replace("\\","/"), index=False)
related_df.to_csv(os.path.join(outdir, 'related_time_series.csv').replace("\\","/"), index = False)

In [ ]:
s3.upload_file(Filename="./forecast-data/target_time_series.csv", Bucket=bucket, Key="{}/{}".format(prefix, 'target_time_series.csv')
)
s3.upload_file(Filename="./forecast-data/related_time_series.csv", Bucket=bucket, Key="{}/{}".format(prefix, 'related_time_series.csv')
)

## Define the dataset schemas to ingest into Forecast

Forecast has a number of predefined **Domains** which come with predefined schemas for data ingestion. Since we are interested in *web traffic*, we choose the WEB_TRAFFIC domain below.

This provides a predefined schema and attribute types for the attributes we include in the target and related time series. For the WEB_TRAFFIC domain, there is no item metadata, only target and related time series data is allowed. 


### Define the schema for the target time series

In [ ]:
# Set the dataset name to a new unique value. If it already exists, go to the Forecast console and delete any existing
# dataset ARNs and datasets.

datasetName = 'webtraffic_forecast_NLP'

schema ={
   "Attributes":[
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },    
       {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"value",
         "AttributeType":"float"
      }      
   ]
}

try:
    response = forecast.create_dataset(
                    Domain="WEB_TRAFFIC",
                    DatasetType='TARGET_TIME_SERIES',
                    DatasetName=datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = schema
                   )
    datasetArn = response['DatasetArn']
    print('Success')
except forecast.exceptions.ResourceAlreadyExistsException as e:
    print(e)
    datasetArn = 'arn:aws:forecast:{}:{}:dataset/{}'.format(REGION_NAME, ACCOUNT_NUM, datasetName)


### Define the schema for the related time series

In [ ]:
# Set the dataset name to a new unique value. If it already exists, go to the Forecast console and delete any existing
# dataset ARNs and datasets.

datasetName = 'webtraffic_forecast_related_NLP'
schema ={
   "Attributes":[{
         "AttributeName":"item_id",
         "AttributeType":"string"
      }, 
       {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
       {
         "AttributeName":"SentimentHeadline",
         "AttributeType":"float"
      }]
    + 
      [{
         "AttributeName":"Headline_{}".format(x),
         "AttributeType":"float"
      } for x in range(10)] 
}

try:
    response=forecast.create_dataset(
                    Domain="WEB_TRAFFIC",
                    DatasetType='RELATED_TIME_SERIES',
                    DatasetName=datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = schema
                   )
    related_datasetArn = response['DatasetArn']
    print('Success')
except forecast.exceptions.ResourceAlreadyExistsException as e:
    print(e)
    related_datasetArn = 'arn:aws:forecast:{}:{}:dataset/{}'.format(REGION_NAME, ACCOUNT_NUM, datasetName)

### Define the dataset group

Before ingesting any data into Forecast we need to combine the target and related time series into a dataset group. We define this below.

In [ ]:
datasetGroupName = 'webtraffic_forecast_NLPgroup'
    
try:
    create_dataset_group_response = forecast.create_dataset_group(DatasetGroupName=datasetGroupName,
                                                              Domain="WEB_TRAFFIC",
                                                              DatasetArns= [datasetArn, related_datasetArn]
                                                             )
    datasetGroupArn = create_dataset_group_response['DatasetGroupArn']
    print('Success')

except forecast.exceptions.ResourceAlreadyExistsException as e:
    print(e)
    datasetGroupArn = 'arn:aws:forecast:{}:{}:dataset-group/{}'.format(REGION_NAME, ACCOUNT_NUM, datasetGroupName)
                                                                                                              

In [ ]:
datasetGroupArn

In [ ]:
forecast.describe_dataset_group(DatasetGroupArn=datasetGroupArn)

## Ingest the target and related time series data from S3

In [ ]:
s3DataPath = 's3://{}/{}/target_time_series.csv'.format(bucket, prefix)
datasetImportJobName = 'forecast_DSIMPORT_JOB_TARGET'

try:
    ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )
    ds_import_job_arn=ds_import_job_response['DatasetImportJobArn']
    target_ds_import_job_arn = copy.copy(ds_import_job_arn) #used to delete the resource during cleanup
except forecast.exceptions.ResourceAlreadyExistsException as e:
    print(e)
    ds_import_job_arn='arn:aws:forecast:{}:{}:dataset-import-job/{}/{}'.format(REGION_NAME, ACCOUNT_NUM, datasetArn, datasetImportJobName)

In [ ]:
#check status -- it will change from IN PROGRESS to ACTIVE once the dataset upload is completed.
while True:
    dataImportStatus = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)['Status']
    print(dataImportStatus)
    if dataImportStatus != 'ACTIVE' and dataImportStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break


In [ ]:
s3DataPath = 's3://{}/{}/related_time_series.csv'.format(bucket, prefix)
datasetImportJobName = 'forecast_DSIMPORT_JOB_RELATED'
try:
    ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=related_datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )
    ds_import_job_arn=ds_import_job_response['DatasetImportJobArn']
    related_ds_import_job_arn = copy.copy(ds_import_job_arn) #used to delete the resource during cleanup
except forecast.exceptions.ResourceAlreadyExistsException as e:
    print(e)
    ds_import_job_arn='arn:aws:forecast:{}:{}:dataset-import-job/{}/{}'.format(REGION_NAME, ACCOUNT_NUM, related_datasetArn, datasetImportJobName)

In [ ]:
while True:
    dataImportStatus = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)['Status']
    print(dataImportStatus)
    if dataImportStatus != 'ACTIVE' and dataImportStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break


## Choose the Model

While DeepAR in SageMaker is a single built-in algorithm for time series analysis, Amazon Forecast provides much greater flexibility with choosing out-of-the-box time series algorithms for model training. Additionally, there is an AutoML feature where you let Amazon Forecast choose the best model based on the data and the weighted average of the p10, p50 and p90 quantile losses.

Here we choose the DeepAR+ algorithm as it is capable of building a global model based on all the different target time series data. Additonally, like the Prophet and NPTS algorithms, DeepAR+ can also incorporate information from the related time series which we provide here. 

For more details on the DeepAR+ algorithm and the differences between this and DeepAR, please see: https://docs.aws.amazon.com/forecast/latest/dg/aws-forecast-recipe-deeparplus.html

Currently only the DeepAR+ algorithm supports hyperparamter optimization. 

In [ ]:
predictorName = 'web_traffic_forecast' + 'DeepARPlus'
forecastHorizon = 15
algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus' 
# choose an algorithm here or set AutoML to be true. Possible algorithmARN choices are:
#ARIMA (no related time series): arn:aws:forecast:::algorithm/ARIMA
#ETS (no related time series) arn:aws:forecast:::algorithm/ETS
#NPTS: arn:aws:forecast:::algorithm/NPTS
#Prophet: arn:aws:forecast:::algorithm/Prophet
 

## Create Predictor

In [ ]:
try:
    create_predictor_response=forecast.create_predictor(PredictorName=predictorName, 
                                                  ForecastHorizon=forecastHorizon,
                                                  AlgorithmArn=algorithmArn,
                                                  PerformAutoML=False, # change to true if want to perform AutoML
                                                  PerformHPO=False, # change to true to perform HPO
                                                  EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                         "BackTestWindowOffset": 15}, 
                                                  InputDataConfig= {"DatasetGroupArn": datasetGroupArn},
                                                  FeaturizationConfig= {"ForecastFrequency": "D", 
                                                                        }
                                                 )
    predictorArn=create_predictor_response['PredictorArn']
except forecast.exceptions.ResourceAlreadyExistsException as e:
    predictorArn = 'arn:aws:forecast:{}:{}:predictor/{}'.format(REGION_NAME, ACCOUNT_NUM, predictorName)

In [ ]:
#note: this will take a few minutes
while True:
    predictorStatus = forecast.describe_predictor(PredictorArn=predictorArn)['Status']
    print(predictorStatus)
    if predictorStatus != 'ACTIVE' and predictorStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break



Describe the Predictor we just created

In [ ]:
forecast.describe_predictor(PredictorArn=predictorArn)


## Metrics from Backtesting

In [ ]:
print('Done creating predictor. Getting accuracy numbers for DeepAR+ ...')

error_metrics_deep_ar_plus = forecast.get_accuracy_metrics(PredictorArn=predictorArn)
error_metrics_deep_ar_plus


In [ ]:
def extract_summary_metrics(metric_response, predictor_name):
    df = pd.DataFrame(metric_response['PredictorEvaluationResults']
                 [0]['TestWindows'][0]['Metrics']['WeightedQuantileLosses'])
    df['Predictor'] = predictor_name
    return df

deep_ar_metrics = extract_summary_metrics(error_metrics_deep_ar_plus, "DeepAR")
pd.concat([deep_ar_metrics]) \
    .pivot(index='Quantile', columns='Predictor', values='LossValue').plot.bar()


## Create a Forecast

In [ ]:
project = "News_Forecast"
print(f"Done fetching accuracy numbers. Creating forecaster for DeepAR+ ...")
forecast_name_deep_ar = f'{project}_deep_ar_plus'


In [ ]:
create_forecast_response_deep_ar = forecast.create_forecast(ForecastName=forecast_name_deep_ar,
                                                        PredictorArn=predictorArn)

In [ ]:
forecast_arn_deep_ar = create_forecast_response_deep_ar['ForecastArn']

In [ ]:
while True:
    forecastStatus = forecast.describe_forecast(ForecastArn=forecast_arn_deep_ar)['Status']
    print(forecastStatus)
    if forecastStatus != 'ACTIVE' and forecastStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break


## Query the Forecast

Having created the forecast, let's now query the results to find out the popularity of the different topics in the original dataset.

In [ ]:
def plot_results(origdf, item_id, decoded_response):
    quantile = [10, 50, 90]
    df = pd.DataFrame()
    origseries= origdf[origdf['Topic'] == int(item_id)].loc[start_training:end_training-dt.timedelta(days=1)].Facebook.tolist()
    index = origdf[origdf['Topic'] == int(item_id)].loc[start_training:end_training-dt.timedelta(days=1)].index.append(pd.date_range(start = end_training-dt.timedelta(days=1), periods= forecastHorizon, freq = '1D'))
    base_series = origseries
    for q in quantile:
        base_series.extend([decoded_response['p{}'.format(q)][x]['Value'] for x in range(forecastHorizon)])
        df['Quantile_{}'.format(q)] = base_series
        base_series = origdf[origdf['Topic'] == int(item_id)].loc[start_training:end_training-dt.timedelta(days=1)].Facebook.tolist()
    
    #adding ground truth to plot
    base_series = origdf[origdf['Topic'] == int(item_id)].loc[start_training:end_GT].Facebook.tolist()
    df['Ground Truth'] = base_series
    
    #reset base_series and index for next iteration
    index = origdf[origdf['Topic'] == int(item_id)].loc[start_training:end_training-dt.timedelta(days=1)].index.append(pd.date_range(start = end_training-dt.timedelta(days=1), periods= forecastHorizon, freq = '1D'))
    base_series = origdf[origdf['Topic'] == int(item_id)].loc[start_training:end_training-dt.timedelta(days=1)].Facebook.tolist()
    
    df['period']=index
    df = df.reset_index().set_index('period')

    return df, df[['Quantile_10','Quantile_50', 'Quantile_90', 'Ground Truth']][-200:].plot(figsize=(15, 4))


In [ ]:
for item_id in range(0, 4):
    forecast_response_deep = forecastquery.query_forecast(
        ForecastArn=forecast_arn_deep_ar,
        Filters={"item_id": str(item_id)})
    df_forecast, plot = plot_results(df, str(item_id), forecast_response_deep['Forecast']['Predictions'])
    if item_id == 1:
        rmse = np.sqrt(mean_squared_error(df_forecast['Ground Truth'], df_forecast['Quantile_50']))


In [ ]:
print("The Root Mean Square Error for the 15 day forecast is {}".format(rmse))

## Defining the Things to Cleanup

#### note: Deleting the Forecast takes a few minutes, but these cleanup sets must be done in order 

In [ ]:
# Delete the Foreacst:
util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn=forecast_arn_deep_ar))

In [ ]:
# Delete the Predictor:
util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn=predictorArn))

In [ ]:
# Delete Import Jobs
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=target_ds_import_job_arn))

In [ ]:
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=related_ds_import_job_arn))

In [ ]:
# Delete the Datasets:
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=datasetArn))

In [ ]:
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=related_datasetArn))

In [ ]:
# Delete the DatasetGroup: datasetGroupArn
util.wait_till_delete(lambda: forecast.delete_dataset_group(DatasetGroupArn=datasetGroupArn))

# Conclusion

In this set of notebooks we showed how to include unstructured text data into your Forecasting use case by leveraging both Amazon SageMaker's built-in DeepAR algorithm and Amazon Forecast which is a fully managed service.

While the datasets used here are merely for illustration purposes, the content of these notebooks can be readily adapted to your particular use cases. Remember that in order to see lift from deep learning models, you almost always need a lot of data; or the models will tend to overfit and not generalize well. 

However most enterprises have a large amount of unstructured data available. By using Neural topic models, relevant semantic information within this unstructured text can be organized into topics, and that topic information can be leveraged as a "feature" input into a time series model.